# Conceptual Design of a Box-Wing Aircraft

In [7]:
from AeronauticConstants import AeronauticConstants as ac
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


The five independent equations are:

$$ x_R =  \frac{V_{stall,\ hyp.}^2}{2 T_0 x_{FR} g} \cdot Q_{MTO} \qquad \text{(Take-Off)} $$

$$ \frac{1}{2} \rho_{0} V_{stall,\ hyp.} ^ 2 S_w C_{L,\ hyp} = Q_{MTO} \qquad \text{(Lift-Off)} $$

$$ x_F = \frac{(V_{stall,\ hyp}^{Land})^2}{2 \bar{\bar a}} \qquad \text{(Braking)} $$

Note: \(V_{stall,\ hyp}^{Land} \ne  V_{stall,\ hyp}\) as they are defined at two different weight conditions (\(Q_{MTO}\) and \(Q_{MTO} - Q_f\)).

$$ (V_{stall,\ hyp}^{Land})^2 = \frac{Q_{MTO} (1 - k)}{\frac{1}{2} \rho_0 S_w C_{L,\ hyp}} \cdot \frac{1}{\bar{\bar a}} \qquad (\text{Landing}) $$

$$ Q_{MTO} = Q_{OE} + Q_{f} + Q_{pay} \qquad (\text{Weights}) $$

Some of the parameters involved are actually from other assumptions or equations (which should increase their number to more than five); however, they are not included here, as these equations relate to the operative aspects of the aircraft and not to its setting (e.g., the autonomy depends on the engine SFC, which is fixed once the engine is fixed; however, the landing distance imposes an external condition on the aircraft setting, which is determined by the fact that the airports are of a certain size).

In [24]:
x_TO = (1300 + 1800) / 2 # [m] Balanced Take-Off Length: from full stop to virtual vertical obstalce at 35 ft (1300 m < x_TO < 1800 m)

z_CR = 20e3 * ac.FT_TO_M # [m] Cruise altitude

x_LD = (1000 + 1500) / 2 # [m] Balanced Landing Length: from V_land = 115 kts to full stop (1000 m < x_TO < 1500 m)

V_land = 115 * ac.KTS_TO_MPS # [m/s]

## $Q_{pay}$

From the Eq. (Weights), we start by evaluating the $Q_{pay}$ contribution, as it is given from the TLARs.

In [25]:
n_pax = 90 # [-] Number of passengers (80 < n_pax < 100)

Q_per_pax = 95 # [kg] mass per passenger

Q_pax = Q_per_pax * n_pax 

# ------ 

n_crew = 4 # [-] Number of company crew members

Q_crew = Q_per_pax * n_crew # [kg] 

# ------

Q_per_seat = 15 # [kg] # mass per seat 

Q_seats = Q_per_seat * n_pax # [kg]

# ------

Q_liq_0 = 200 # [kg]

Q_liq_add_per_pax = 1.9 # [kg]

Q_liq = Q_liq_0 + Q_liq_add_per_pax * n_pax # [kg]

# ------

V_per_lugg = 5 * (ac.FT_TO_M) ** 3 # [m^3]

V_lugg = V_per_lugg * n_pax # [m^3]

# ------

Q_pay = Q_pax + Q_crew + Q_seats + Q_liq

print(f"The total mass of payload to be carried is: Q_pay = {Q_pay} kg")

The total mass of payload to be carried is: Q_pay = 10651.0 kg



From CS 25.1 Applicability:

(a) These Certification Specifications are applicable to turbine powered Large Aeroplanes.

From wikipedia:

"This certification procedure applies to large, turbine-powered aircraft, with max take-off weight more than 5,700kg (CS 25.1)"

So we are in this Aircraft Category

## $ Q_{OE} $ 
We already determined $Q_{pay}$, the next step is to determine $Q_{EO}$. We will do so thanks to statistical formulae available in textbooks. In order for them to be valid, we have to make some adjustments and carefully check the founding hypothesis to ensure that they will work for our configuration (box-wing) as well.

In [ ]:
# Q_MTO = np.linspace(30e3, 50e3, 20) # [kg] Maximum Take-Off Mass
Q_MTO = 40e3

def ratio_Q_OE_estimate(Q_MTO, A = 0.97, C = -0.06):
    ratio_Q_OE = A * Q_MTO ** C
    return ratio_Q_OE

Q_OE = Q_MTO * ratio_Q_OE_estimate(Q_MTO)

## Autonomy range calculation

In [ ]:
# Initial guess for Q_MTO
Q_MTO_start = 80e3  # [kg] initial guess
tolerance = 100  # Convergence criterion [kg]
max_iter = 100
iteration = 0

Q_f_non_usable = 0.003
ratio_Q_f_TO = 0.97
ratio_Q_f_CL = 0.985
ratio_Q_f_LG = 0.995
E_CR = 17
mach_CR = 0.8

# Load SFC data once
data_sfc = pd.read_csv("high_bpr_sfc_vs_mach.csv")
mach = data_sfc["mach"]
sfc = data_sfc["sfc"]
sfc_interp = interp1d(mach, sfc)
mach_dense = np.linspace(mach.min(), mach.max(), 100)
sfc_dense = sfc_interp(mach_dense)
idx_SFC_CR = np.argmin(np.abs(mach_CR - mach_dense))
SFC_CR = sfc_dense[idx_SFC_CR] * (1 / ac.H_TO_S)  # [1/s]

# Cruise speed
T_CR = ac.T_isa(z_CR)
V_CR = mach_CR * ac.c_sound(T_CR)
x_range = 1000 * ac.NM_TO_KM * 1000  # [m] Required range

# Iterative loop

Q_MTO = Q_MTO_start
while iteration < max_iter:
    iteration += 1
    
    # Estimate OE mass
    Q_OE = Q_MTO * ratio_Q_OE_estimate(Q_MTO)
    
    # Fuel fraction for cruise
    ratio_Q_f_CR = np.exp(-SFC_CR * x_range / (E_CR * V_CR))
    ratio_Q_f_mission = ratio_Q_f_TO * ratio_Q_f_CL * ratio_Q_f_CR * ratio_Q_f_LG * (1 - Q_f_non_usable)
    
    # Fuel mass
    Q_f = ratio_Q_f_mission * Q_MTO
    
    # Total aircraft mass for next iteration
    Q_MTO_new = Q_f + Q_pay + Q_OE

    print(f"{Q_MTO_new:.4e}")
    
    # Check convergence
    if abs(Q_MTO_new - Q_MTO) < tolerance:
        print(f"Converged after {iteration} iterations.")
        break
    
    # Update Q_MTO for next iteration
    Q_MTO = Q_MTO_new

# # Final results
# print(f"Starting Q_MTO = {Q_MTO_start:.2e} kg")
# print(f"Starting Q_pay = {Q_pay:.2e} kg")
# print(f"Final Q_f = {Q_f:.2e} kg")
# print(f"Final Q_OE = {Q_OE:.2e} kg")
# print(f"Total aircraft mass = {Q_f + Q_pay + Q_OE:.2e} kg")




1.2049e+05
1.7463e+05
2.4648e+05
3.4113e+05
4.6500e+05
6.2612e+05
8.3446e+05
1.1024e+06
1.4451e+06
1.8812e+06
2.4335e+06
3.1296e+06
4.0029e+06
5.0938e+06
6.4506e+06
8.1311e+06
1.0204e+07
1.2752e+07
1.5870e+07
1.9673e+07
2.4294e+07
2.9890e+07
3.6643e+07
4.4767e+07
5.4506e+07
6.6146e+07
8.0016e+07
9.6492e+07
1.1601e+08
1.3906e+08
1.6621e+08
1.9810e+08
2.3546e+08
2.7911e+08
3.3000e+08
3.8916e+08
4.5778e+08
5.3719e+08
6.2886e+08
7.3446e+08
8.5583e+08
9.9502e+08
1.1543e+09
1.3363e+09
1.5436e+09
1.7795e+09
2.0474e+09
2.3509e+09
2.6942e+09
3.0817e+09
3.5185e+09
4.0100e+09
4.5619e+09
5.1808e+09
5.8735e+09
6.6478e+09
7.5117e+09
8.4741e+09
9.5448e+09
1.0734e+10
1.2053e+10
1.3514e+10
1.5129e+10
1.6913e+10
1.8881e+10
2.1049e+10
2.3433e+10
2.6052e+10
2.8925e+10
3.2074e+10
3.5521e+10
3.9288e+10
4.3402e+10
4.7888e+10
5.2775e+10
5.8093e+10
6.3874e+10
7.0149e+10
7.6956e+10
8.4331e+10
9.2312e+10
1.0094e+11
1.1026e+11
1.2032e+11
1.3117e+11
1.4285e+11
1.5541e+11
1.6892e+11
1.8343e+11
1.9900e+11
2.1570e+11